In [1]:
import pandas as pd
import numpy as np

import nltk
from nltk import tokenize
from nltk.corpus import stopwords

import re
import pymorphy2

import itertools

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV, train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

import pyLDAvis.lda_model

from tqdm import tqdm_notebook
import warnings
import joblib

from utils import *

warnings.filterwarnings('ignore')

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dmitry/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/dmitry/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
df = pd.read_csv('data/topic_modeling_task_sample_trainPart.csv') 

In [3]:
df['replicas'] = df['text_employer'].str.split('.')
data = df[['ucid','replicas']].explode('replicas').reset_index(drop=True).dropna()
data = data[data['replicas'] != '']

In [4]:
stop_words = stopwords.words('russian')
extra_stop_words = get_stopwords_from_file('stopwords.txt')
stop_words_extended = set(stop_words + extra_stop_words)

morph = pymorphy2.MorphAnalyzer()

def process_string(input_string, morph=morph, stop_words=stop_words_extended):
    words = re.findall(r'\b[а-яА-Я_]+\b', input_string)
    result_string = ' '.join([word if '_' in word else morph.parse(word)[0].normal_form for word in words if word not in stop_words])
    
    return result_string

data['replicas_processed'] = data['replicas'].apply(process_string)

In [5]:
X_train, X_test = train_test_split(data.replicas, test_size=0.5, random_state=42)

# LDA

In [6]:
vector_ben = CountVectorizer(
    analyzer='word',
    min_df=10,
    ngram_range=(1, 2),
    stop_words=stopwords.words('russian'),
    # max_features=10000,
)
train_vec_ben = vector_ben.fit_transform(X_train)
test_vec_ben = vector_ben.transform(X_test)
train_vec_ben.shape

(278107, 29171)

In [7]:
lda_model_ben = LatentDirichletAllocation(
        n_components=15,
        learning_method='online',
        random_state=42,
        max_iter=10,
        n_jobs=4,
        verbose=1
    )

with joblib.parallel_backend(backend='loky', n_jobs=4):
    lda_model_ben.fit(train_vec_ben)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [8]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.lda_model.prepare(
    lda_model_ben,
    train_vec_ben,
    vector_ben,
    mds='tsne'
)
panel

PreparedData(topic_coordinates=               x          y  topics  cluster      Freq
topic                                                 
12     20.571108  22.230431       1        1  9.155494
13      3.463959  31.040449       2        1  9.126701
10     48.187183  -5.170693       3        1  8.827142
5      30.727898 -11.740621       4        1  8.686411
1      18.292423  42.346806       5        1  8.451303
14     34.522896  30.875523       6        1  6.868457
8       9.742185  -8.400470       7        1  6.634294
2      35.150593  50.686394       8        1  6.576132
3       2.920772  11.607692       9        1  6.505336
6      21.447004   5.424685      10        1  5.627774
9      61.491974   7.253708      11        1  5.502981
0      36.985806   9.853289      12        1  4.904000
11     51.615631  40.908237      13        1  4.826581
7      48.405083  21.196606      14        1  4.177914
4      65.962402  27.258892      15        1  4.129481, topic_info=              Term          Freq         Total Category  logprob  loglift
5256   добрый_день  16765.000000  16765.000000  Default  30.0000  30.0000
786         бизнес  36864.000000  36864.000000  Default  29.0000  29.0000
26819       удобно  23304.000000  23304.000000  Default  28.0000  28.0000
2964       вопросы  16009.000000  16009.000000  Default  27.0000  27.0000
22693         сбер  16954.000000  16954.000000  Default  26.0000  26.0000
...            ...           ...           ...      ...      ...      ...
28659          это   2056.508467  33848.425652  Topic15  -4.2813   0.3861
8528         карта   1203.106930  14279.742581  Topic15  -4.8174   0.7131
13436   необходимо   1008.469553   7053.792797  Topic15  -4.9939   1.2419
17805   подключить    969.585124   8131.115194  Topic15  -5.0332   1.0604
7891     интересно    850.475634   1624.637058  Topic15  -5.1643   2.5398

[708 rows x 6 columns], token_table=       Topic      Freq                       Term
term                                             
58        14  0.998753                      адрес
116        6  0.141117                  актуально
116        8  0.858169                  актуально
143        3  0.742118                      акции
143        5  0.257571                      акции
...      ...       ...                        ...
28902     12  0.998635  это персональный_менеджер
28968      9  0.999564                это связано
28971      7  0.999650                это сделать
29051     10  0.999682                      этому
29144      1  0.999557                    являюсь

[875 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[13, 14, 11, 6, 2, 15, 9, 3, 4, 7, 10, 1, 12, 8, 5])

# Идеи

**Вообще** большая часть экспериментов пропушена. 
В них я менял параметры CountVectorizer и LatentDirichletAllocation и вот какие сделал выводы:
1. Кажется что нормализация может навредить в определнии некоторых категорий, например Призыв к действию
2. Имеет смысл использовать биграммы и триграммы
3. Увеличение количества топиков (до 30) позволяет выявить более разнообразные кластеры, из которых можно черпать темы, отличные от заданных
4. Некоторые топики прослеживаются от запуска к заупску, но не вместе:
    - Приветствие
    - Окончание разговора
    - Инструкция по подключению/приобретению
    - тарифы / цены
    - Вовлечение (подведение к диалогу)
    - Следующие шаги

5. Есть топики, которые классическими методами, кажется, совсем трудно уловить, например
- Болтовня — его скорее стоит использовать, когда другие топики сложно определить, то есть совокупный класс для всего, что не подошло
- Призыв к действию — здесь, возможно, нормализация вообще сделает только хуже, призыв к действию я бы обнаруживал по наличию повелительной формы глаголов
- Вопрос клиента — совсем непонятно, реплик клиента нет, реплика может быть ответом на вопрос клиента, но ее можно классифицировать как инструкцию, например

6. Некоторые топики могут сильно пересекаться:
- Преимущества, Удобство работы, Тарифы / Цены

7. LatentDirichletAllocation из sklearn не параллелится в несколько потоков нормально, все потоки, кроме главного, используют максимум 5% мощности (не помогает и joblib), из-за чего проведение экспериментов занимает слишком много времени.

### Что-то кроме Topic Modeling

1. Самое базовое: проверить наличие ключевых слов в репликах, например для Акции — акция, специальный предложение, Безопасность — безопасность, безопасно, надежно и т.д. Можно для каждой категории составить список слов.
2. Использовать для векторизации что-то более сложное, например, Word2Vec для улавливания семантической близости
3. Суммаризация текста с помощью больших языковых моделей и рассчитывание косинусного расстояния между эмбедингом реплики и эмбедингом каждого топика — выбрать тот, у которого наименьшее расстояние.
4. Можно для каждого топика выбрать несколько реплик 3-7 (автоматически по вхождению слов или вручную) и использовать метрические методы для определения близости.
5. Таким образом можно использовать LDA для выявления новых топиков, а классификацию делать с помощью метрических методов

**Итог**: Последние две идеи кажутся очень перспективными. Вообще, полученна, жаль не хватает времени, чтобы все проверить